In [28]:
import quinn
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
dataset_df = spark.read.csv('Monkey_Pox_Cases_Worldwide.csv', header=True)
daily_df = spark.read.csv(
    'Daily_Country_Wise_Confirmed_Cases.csv', header=True)

total_df = dataset_df.select(
    dataset_df['Country'], dataset_df['Confirmed_Cases'])

dataset_df = total_df.join(daily_df, ['Country'], how='left')


def dashes_to_underscores(s):
    return s.replace("-", "_")


dataset_df = dataset_df.transform(
    quinn.with_columns_renamed(dashes_to_underscores))

dataset_df.write.option('header', True) \
                .csv('../data/daily_cases.csv')


22/07/06 16:03:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [27]:
from pyspark.sql import SparkSession

from google.cloud import bigquery


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/fossa/data/dont_worry_about_it/dearHenry.json"

client = bigquery.Client()
# dataset_id = f"{client.project}.monkeypox"
# bucket = 'hole-in-the'
# spark.conf.set('temporaryGcsBucket', bucket)

# dataset_df.write.format("bigquery") \
#     .mode("overwrite") \
#     .option("temporaryGcsBucket", "databricks-bigquery-temp") \
#     .option("table", "dearliza.monkeypox_data.monkeypox") \
#     .save()

dataset_df.write \
    .format("bigquery") \
    .option("temporaryGcsBucket", "hole-in-the") \
    .option("table", "monkeypox.monkeypox_table")





In [25]:
# Imports the Google Cloud client library
import os
from google.cloud import storage

# Instantiates a client

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/fossa/data/dont_worry_about_it/dearHenry.json"
client = storage.Client()


# Retrieve an existing bucket
# https://console.cloud.google.com/storage/browser/hole-in-the/
bucket = client.get_bucket('hole-in-the')
# Then do other things...
blob = bucket.blob('daily_cases.csv')
blob.upload_from_filename('../data/Daily_Country_Wise_Confirmed_Cases.csv')
blob = bucket.blob('cases_worldwide.csv')
blob.upload_from_filename('../data/Monkey_Pox_Cases_Worldwide.csv')


In [28]:
from google.cloud import bigquery
import pandas as pd 
import pandas_gbq

daily_cases = pd.read_csv('../data/Daily_Country_Wise_Confirmed_Cases.csv')
# cases_worldwide = pd.read_csv('../data/Monkey_Pox_Cases_Worldwide.csv')


client = bigquery.Client()
dataset_id = f"{client.project}.monkeypox"
dataset = bigquery.Dataset(dataset_id)
dataset.location = 'us'
dataset = client.create_dataset(dataset, exists_ok = True, timeout =100)
project_id = 'dearliza'
table_id = 'monkeypox.monkeypox_data'

pandas_gbq.to_gbq(daily_cases, table_id, project_id = project_id, if_exists = 'replace', api_method = 'load_csv')

GenericGBQException: Reason: 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/dearliza/datasets/monkeypox/tables?prettyPrint=false: Invalid field name "2022-05-06". Fields must contain only letters, numbers, and underscores, start with a letter or underscore, and be at most 300 characters long.

In [25]:
import gcloud
from gcloud import dataproc

REGION='us'
CLUSTER_NAME='monkeypox'
gcloud dataproc clusters create ${CLUSTER_NAME} \
    --region ${REGION} \
    --initialization-actions gs://goog-dataproc-initialization-actions-${REGION}/connectors/connectors.sh \
    --metadata bigquery-connector-version=1.2.0 \
    --metadata spark-bigquery-connector-version=0.21.0

SyntaxError: invalid syntax (3079980177.py, line 6)